<a href="https://colab.research.google.com/github/qiaojunch/CS3700-Networks-and-Distributed-Systems/blob/master/assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 - Simple Recommend-Friend Algorithm


In [72]:
import numpy as np
import matplotlib.pyplot as plt

In [73]:
# Get data from txt file
data_path = '/content/soc-LiveJournal1Adj.txt'


In [74]:

def read_txt_file(file_path):
    user_data = {}  # Dictionary to store UserID as int and User's friends as list of int

    try:
        with open(file_path, 'r') as file:
            for line in file:
                parts = line.strip().split('\t')

                # Check informatted line input
                if len(parts) < 2 or parts[0] == '':
                  continue
                user_id = int(parts[0])
                friends = [int(friend) for friend in parts[1].split(',') if friend != '']
                user_data[user_id] = friends
    except Exception as e:
        print(f"An error occurred: {e}")

    return user_data


In [59]:
user_data = read_txt_file(data_path)
print(len(user_data))


49124


In [83]:
from typing import DefaultDict

def recommend_friends(user_data, top_n=10):
    """
    user_data: map {user_id, [other_user1, other_user2, ...]}
    return recommendations: {user1:[other1, other2, ..., otherN]], user2:[...], ...}
    """
    recommendations = DefaultDict(list)

    for user_id, user_friends in user_data.items():
      ## get friend's mutual friends
      # 1: [2, 3, 5]
      mutual_friends_list = []
      for other_id, other_friends in user_data.items():
          # Find friend in common for two users if they are not friend
          if other_id != user_id and other_id not in user_friends:
            mutual_friends = set(user_friends).intersection(other_friends)
            mutual_friends_count = len(mutual_friends)
            if mutual_friends_count == 0:
              continue
            mutual_friends_list.append((mutual_friends_count, other_id))


      ## sort by number of mutual friends and get the topN recommendations
      ## mutual friends: [(num_mutual_friends1, other1), (num_mutual_friends2, other2), ...]
      sort_mutual_friends = sorted(mutual_friends_list, key=lambda x: x[0], reverse=True)[:top_n]

      ## collect mutual friends for each user in data
      recommendations[user_id] = [x[1] for x in sort_mutual_friends]

    return recommendations


In [ ]:
# output format: <usr_id><tab><friend_list>
# test_data = read_txt_file("/content/test.txt")

simple_recomm = recommend_friends(user_data)
ids = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]

# ids = [1, 2, 3, 4, 5, 6, 10]
for id in ids:
  recomm_str = ','.join([str(item) for item in simple_recomm[id]])
  print(f"{id}\t{recomm_str}")


## 2 - Recommend-Friend Algo using PySpark

In [ ]:
#@title Download and install the PySpark packages

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Description of mapReduce algo:

In a Spark-based solution for this problem, I would use the distributed computing capabilities of Apache Spark to efficiently process the large dataset. I would load the data into a Spark DataFrame, perform necessary transformations to calculate mutual friends, and then use Spark's built-in functions for sorting and selecting the top N recommendations for each user. The parallelized nature of Spark allows for scalability and efficient processing of the algorithm across a cluster of machines.

In [65]:
from pyspark import SparkContext

In [66]:
def get_relations(line):
  parts = line.split('\t')

  # check incorrect formatted line or no frirends
  if len(parts) < 2 or parts[0] == '':
    return []
  person = int(parts[0])
  friends = parts[1].split(',')

  # map the relation of a,b to 1 is they are friend, or 0 if not
  # example: ((a, b), 0) => a, b are friends
  #          ((a, c), 1) => a, c are not friends
  friends_list = [((person, int(friend)), 0) for friend in friends if friend != '']
  mutual_friend_list = []

  for i in range(0, len(friends)-1):
    for j in range(i+1, len(friends)):
      a, b = int(friends[i]), int(friends[j])
      mutual_friend_list.append(((a, b), 1))
      mutual_friend_list.append(((b, a), 1))

  return friends_list + mutual_friend_list

In [67]:
def recommend_new_friends(person_and_strangers, n=10):
  """
  person_and_strangers: (p, [(s1, num_mutual_friends), (s2, num_mutual_friends),...(sN, num_mutual_friends))
  n: number of friends to recommend
  return: a list of up to n strangers to recommend as friends
  """

  person, list_of_strangers = person_and_strangers

  # Sort the list_of_strangers by top n mutual friends with person
  # x: (stranger, num_mutual_friends)
  ordered_list_of_strangers = sorted(list_of_strangers,
                                     key=lambda x: (-x[1], x[0]))[:n]

  # Pass ordered_list_of_strangers to lambda function and return strangers list who has top-N mutual friends
  # x: (stranger, num_mutual_friends)
  recommendations = map(lambda x: x[0], ordered_list_of_strangers)

  return person, list(recommendations)

In [68]:
# Initialize spark configuration and context
sc = SparkContext(appName="recommend-friends")

# Parse data and create relationship map
data_rdd = sc.textFile(data_path)
relationship_pairs = data_rdd.flatMap(lambda line: get_relations(line))
already_friends = relationship_pairs.filter(lambda x: x[1] == 0)

# pair_and_count: [((p1, s1), num_of_mutual_friends), ...]
pair_and_count = relationship_pairs.subtractByKey(already_friends).reduceByKey(lambda a, b: a + b)

# person_and_friends: [(p1, (s1, num_of_mutual_friens)), (p2, (s2, num_of_mutual_friends)), ...]
person_and_strangers = pair_and_count.map(lambda x: (x[0][0], (x[0][1], x[1]))) \
                        .groupByKey() \
                        .mapValues(list)

# Find recommendations for person
recommendations = person_and_strangers.map(lambda x: recommend_new_friends(x))


# Export output
# recommendations: [(person1, recommendation_list1), (person2, recommendaton_list2), ...]
recommendations \
  .map(lambda item: f"{item[0]}\t{','.join(map(lambda x: str(x), item[1]))}") \
  .saveAsTextFile("/content/output")

sc.stop()

After done with recommendations algorithm, we want to save the result as output.txt file, and print out the recommendations result for a specific person list.

In [69]:
path1 = "/content/output/part-00000"
path2 = "/content/output/part-00001"
path3 = "/content/output/part-00003"

outputName = "output.txt"
inputNames = [path1, path2, path3]
friends_map = DefaultDict(str)


def parse_file(line):
  parts = line.split("\t")

  # check incorrect formatted line or no frirends
  if len(parts) < 2 or parts[0] == "":
    return []
  person = int(parts[0])
  friends = parts[1]

  return person, friends


for inputName in inputNames:
  with open(inputName, "r") as inFile:

    for line in inFile.readlines():
      person, friends = parse_file(line)
      friends_map[person] = friends

# Sort all input by person ID
sort_friends_map = sorted(friends_map.items())

# Write sorted friends_map to a output file
with open(outputName, "w") as outFile:

  for item in sort_friends_map:
    line = f"{item[0]}\t{item[1]}"
    outFile.write(line)

# Print test result
person_list = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]
for p in person_list:
  print(f"{p}\t{friends_map[p]}")


924	439,2409,6995,11860,15416,43748,45881

8941	8943,8944,8940

8942	
9019	9022,317,9023

9020	9021,9016,9017,9022,317,9023

9021	9020,9016,9017,9022,317,9023

9022	
9990	
9992	9987,9989,35667,9991

9993	9991,13134,13478,13877,34299,34485,34642,37941

